In [2]:
'''Getting article from any news site using ScrapeArticle Class '''
import NewsScraper as ns
obj = ns.ScrapeArticle()
obj.get_news_from_url("https://timesofindia.indiatimes.com/world")
articles = obj.get_article()
print(articles)

                                            Keywords  \
0  [korea, north, maintains, kim, health, jong, s...   
1  [covid19, facebook, way, videos, mayor, reside...   
2  [covid19, worsened, work, recovering, returns,...   
3  [say, officers, louisiana, law, shooting, offi...   
4  [covid19, clean, stay, turning, reveals, india...   

                                             Summary  \
0  FILE - North Korean leader Kim Jong Un.\n(AP p...   
1  Covid-19: Mayor entertains residents after imp...   
2  Covid-19: UK PM Boris Johnson returns to work ...   
3  1 officer dead, 1 wounded in Louisiana shootin...   
4  Covid-19: Lockdown reveals fresh and clean air...   

                                                Text  \
0  FILE - North Korean leader Kim Jong Un. (AP ph...   
1  Covid-19: Mayor entertains residents after imp...   
2  Covid-19: UK PM Boris Johnson returns to work ...   
3  1 officer dead, 1 wounded in Louisiana shootin...   
4  Covid-19: Lockdown reveals fresh and clean 

In [3]:
'''Preprocessing the scraped article usinf PreprocessData class'''
import preprocess_data as ps
ob = ps.PreprocessData()
ob.preprocessor(articles)

In [4]:
'''converting the final data into proper dataframe to be tested'''
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from textblob import TextBlob

df=[]
for article in articles:
    
    pred_info={} 
    article.parse()
    analysis=TextBlob(article.text)
    polarity=analysis.sentiment.polarity
    title_analysis=TextBlob(article.title)
    pred_info['title']=article.title
    pred_info['text']=article.text
    pred_info['n_tokens_title']=len(word_tokenize(article.title))
    pred_info['n_tokens_content']=len(word_tokenize(article.text))
    pred_info['n_unique_tokens']=ob.rate_unique(article.text)
    pred_info['n_non_stop_words']=ob.rate_nonstop(article.text)[0]
    pred_info['n_non_stop_unique_tokens']=ob.rate_nonstop(article.text)[1]
    pred_info['num_hrefs']=article.html.count("https://timesofindia.indiatimes.com")
    pred_info['num_imgs']=len(article.images)
    pred_info['num_videos']=len(article.movies)
    pred_info['average_token_length']=ob.avg_token(article.text)
    pred_info['num_keywords']=len(article.keywords)
    
    if "life-style" in article.url:
        pred_info['data_channel_is_lifestyle']=1
    else:
        pred_info['data_channel_is_lifestyle']=0
    if "etimes" in article.url:
        pred_info['data_channel_is_entertainment']=1
    else:
        pred_info['data_channel_is_entertainment']=0
    if "business" in article.url:
        pred_info['data_channel_is_bus']=1
    else:
        pred_info['data_channel_is_bus']=0
    if "social media" or "facebook" or "whatsapp" in article.text.lower():
        data_channel_is_socmed=1
        data_channel_is_tech=0
        data_channel_is_world=0
    else:
        data_channel_is_socmed=0
    if ("technology" or "tech" in article.text.lower()) or ("technology" or "tech" in article.url):
        data_channel_is_tech=1
        data_channel_is_socmed=0
        data_channel_is_world=0
    else:
        data_channel_is_tech=0
    if "world" in article.url:
        data_channel_is_world=1
        data_channel_is_tech=0
        data_channel_is_socmed=0
    else:
        data_channel_is_world=0
        
    pred_info['data_channel_is_socmed']=data_channel_is_socmed
    pred_info['data_channel_is_tech']=data_channel_is_tech
    pred_info['data_channel_is_world']=data_channel_is_world
    if ob.day(article.text)=="Monday":
        pred_info['weekday_is_monday']=1
    else:
        pred_info['weekday_is_monday']=0
    if ob.day(article.text)=="Tuesday":
        pred_info['weekday_is_tuesday']=1
    else:
        pred_info['weekday_is_tuesday']=0
    if ob.day(article.text)=="Wednesday":
        pred_info['weekday_is_wednesday']=1
    else:
        pred_info['weekday_is_wednesday']=0
    if ob.day(article.text)=="Thursday":
        pred_info['weekday_is_thursday']=1
    else:
        pred_info['weekday_is_thursday']=0
    if ob.day(article.text)=="Friday":
        pred_info['weekday_is_friday']=1
    else:
        pred_info['weekday_is_friday']=0
    if ob.day(article.text)=="Saturday":
        pred_info['weekday_is_saturday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_saturday']=0
    if ob.day(article.text)=="Sunday":
        pred_info['weekday_is_sunday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_sunday']=0
        pred_info['is_weekend']=0
        
    
    pred_info['global_subjectivity']=analysis.sentiment.subjectivity
    pred_info['global_sentiment_polarity']=analysis.sentiment.polarity
    pred_info['global_rate_positive_words']=ob.rates(article.text)[0]
    pred_info['global_rate_negative_words']=ob.rates(article.text)[1]
    pred_info['avg_positive_polarity']=ob.rates(article.text)[2]
    pred_info['min_positive_polarity']=ob.rates(article.text)[3]
    pred_info['max_positive_polarity']=ob.rates(article.text)[4]
    pred_info['avg_negative_polarity']=ob.rates(article.text)[5]
    pred_info['min_negative_polarity']=ob.rates(article.text)[6]
    pred_info['max_negative_polarity']=ob.rates(article.text)[7]    
    pred_info['title_subjectivity']=title_analysis.sentiment.subjectivity
    pred_info['title_sentiment_polarity']=title_analysis.sentiment.polarity
    df.append(pred_info)

In [14]:
'''visulaization of dataframe'''
import pandas as pd
pred_df=pd.DataFrame(df)
pred_test=pred_df.drop(['text'],axis=1)
pred_test=pred_test.drop(['title'],axis=1)
pred_df.head()

,average_token_length,avg_negative_polarity,avg_positive_polarity,data_channel_is_bus,data_channel_is_entertainment,data_channel_is_lifestyle,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,global_rate_negative_words,...,title,title_sentiment_polarity,title_subjectivity,weekday_is_friday,weekday_is_monday,weekday_is_saturday,weekday_is_sunday,weekday_is_thursday,weekday_is_tuesday,weekday_is_wednesday
0,4.665301,-0.237037,0.301190,0,0,0,0,0,1,0.045,...,Kim Jong Un Health: South Korea maintains Kim ...,0.000000,0.0,0,1,0,0,0,0,0
1,3.869110,0.000000,0.400000,0,0,0,0,0,1,0.000,...,Covid-19: Mayor entertains residents after imp...,0.000000,0.0,0,1,0,0,0,0,0
2,4.640777,-0.441667,0.100000,0,0,0,0,0,1,0.015,...,UK PM Boris Johnson returns to work after reco...,0.000000,0.0,0,0,0,1,0,0,0
3,4.393548,-0.200000,0.200000,0,0,0,0,0,1,0.010,...,"1 officer dead, 1 wounded in Louisiana shooting",-0.200000,0.4,0,0,0,0,0,0,1
4,4.543307,-0.233333,0.307008,0,0,0,0,0,1,0.015,...,Covid-19: Lockdown reveals fresh and clean air...,0.333333,0.6,0,0,0,1,0,0,0


In [24]:
'''loading the saved model and predicting'''
import pickle 
with open('saved_model.pkl', 'rb') as file:  
    loaded_model = pickle.load(file)
    out=pd.DataFrame(loaded_model.predict(pred_test),pred_df['title'])
    out.reset_index(level=0, inplace=True)
    out = out.rename(index=str, columns={"index": "News", 0: "Virality"})

In [25]:
print(out)

                                                title  Virality
0   Kim Jong Un Health: South Korea maintains Kim ...  15608.44
1   Covid-19: Mayor entertains residents after imp...  14547.20
2   UK PM Boris Johnson returns to work after reco...  18172.56
3     1 officer dead, 1 wounded in Louisiana shooting  24734.14
4   Covid-19: Lockdown reveals fresh and clean air...  24652.32
5             Iran's Guard says it launched satellite  27413.08
6   Covid-19: Virus shutdown shows human impact on...  18888.56
7   San Francisco: Dog stolen in December has joyo...  17790.80
8   Crude Oil prices drop to historic lows after e...  22340.14
9   Covid-19: Los Angeles study indicates higher i...  21576.32
10  Covid-19: President Trump says 'politics' is b...  20107.66
11  US federal coronavirus plan is clear and scien...  18778.46
12  Balakot strike, abrogation of Article 370 to h...  14005.22
13  Covid-19: India hands over HCQ tablets, emerge...  15572.56
14  Boris Johnson back at work, warns ag

In [26]:
out.to_csv('predicted_virality.csv')

In [28]:
sort_by_virality = out.sort_values('Virality',ascending=False)

In [29]:
print(sort_by_virality)

                                                title  Virality
5             Iran's Guard says it launched satellite  27413.08
20  Trump nominates Indian-American as US envoy to...  24806.14
3     1 officer dead, 1 wounded in Louisiana shooting  24734.14
4   Covid-19: Lockdown reveals fresh and clean air...  24652.32
8   Crude Oil prices drop to historic lows after e...  22340.14
9   Covid-19: Los Angeles study indicates higher i...  21576.32
30  Star Talk: Radhika Apte opens up about learnin...  20822.32
10  Covid-19: President Trump says 'politics' is b...  20107.66
6   Covid-19: Virus shutdown shows human impact on...  18888.56
28  Coronavirus in Noida: Noida registers 14 new c...  18870.32
11  US federal coronavirus plan is clear and scien...  18778.46
32  Covid-19 lockdown: This UP cop sings to warn v...  18304.56
31  Narendra Modi: PM Modi interacts with CMs via ...  18172.56
2   UK PM Boris Johnson returns to work after reco...  18172.56
7   San Francisco: Dog stolen in Decembe

In [30]:
sort_by_virality.to_csv('sort_by_virality.csv')